In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import metrics

## Loading Data

In [2]:
import chardet    
rawdata = open(r'SCDB_2021_01_caseCentered_Citation.csv', 'rb').read()
result = chardet.detect(rawdata)
charenc = result['encoding']
print(charenc)

Windows-1252


In [3]:
cases = pd.read_csv(r'SCDB_2021_01_caseCentered_Citation.csv', encoding=charenc)
pd.options.display.max_columns = None
display(cases)
cases.shape

,caseId,docketId,caseIssuesId,voteId,dateDecision,decisionType,usCite,sctCite,ledCite,lexisCite,term,naturalCourt,chief,docket,caseName,dateArgument,dateRearg,petitioner,petitionerState,respondent,respondentState,jurisdiction,adminAction,adminActionState,threeJudgeFdc,caseOrigin,caseOriginState,caseSource,caseSourceState,lcDisagreement,certReason,lcDisposition,lcDispositionDirection,declarationUncon,caseDisposition,caseDispositionUnusual,partyWinning,precedentAlteration,voteUnclear,issue,issueArea,decisionDirection,decisionDirectionDissent,authorityDecision1,authorityDecision2,lawType,lawSupp,lawMinor,majOpinWriter,majOpinAssigner,splitVote,majVotes,minVotes
0,1946-001,1946-001-01,1946-001-01-01,1946-001-01-01-01,11/18/1946,1,329 U.S. 1,67 S. Ct. 6,91 L. Ed. 3,1946 U.S. LEXIS 1724,1946,1301,Vinson,24,HALLIBURTON OIL WELL CEMENTING CO. v. WALKER e...,1/9/1946,10/23/1946,198,NaN,172.0,NaN,6.0,NaN,NaN,0.0,51.0,6.0,29.0,NaN,0.0,11.0,2.0,1.0,1.0,3.0,0.0,1.0,1.0,0.0,80180.0,8.0,2.0,0.0,4.0,NaN,6.0,600.0,35 U.S.C. § 33,78.0,78.0,1,8,1
1,1946-002,1946-002-01,1946-002-01-01,1946-002-01-01-01,11/18/1946,1,329 U.S. 14,67 S. Ct. 13,91 L. Ed. 12,1946 U.S. LEXIS 1725,1946,1301,Vinson,12,CLEVELAND v. UNITED STATES,10/10/1945,10/17/1946,100,NaN,27.0,NaN,1.0,NaN,NaN,0.0,123.0,52.0,30.0,NaN,0.0,4.0,2.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0,10500.0,1.0,1.0,0.0,4.0,NaN,6.0,600.0,18 U.S.C. § 398,81.0,87.0,1,6,3
2,1946-003,1946-003-01,1946-003-01-01,1946-003-01-01-01,11/18/1946,1,329 U.S. 29,67 S. Ct. 1,91 L. Ed. 22,1946 U.S. LEXIS 3037,1946,1301,Vinson,21,CHAMPLIN REFINING CO. v. UNITED STATES ET AL.,11/8/1945,10/18/1946,209,NaN,27.0,NaN,2.0,66.0,NaN,1.0,107.0,42.0,107.0,42.0,0.0,1.0,NaN,2.0,1.0,2.0,0.0,0.0,0.0,0.0,80250.0,8.0,2.0,0.0,1.0,NaN,2.0,207.0,NaN,84.0,78.0,1,5,4
3,1946-004,1946-004-01,1946-004-01-01,1946-004-01-01-01,11/25/1946,7,329 U.S. 40,67 S. Ct. 167,91 L. Ed. 29,1946 U.S. LEXIS 1696,1946,1301,Vinson,26,UNITED STATES v. ALCEA BAND OF TILLAMOOKS ET AL.,1/31/1946,10/25/1946,27,NaN,170.0,NaN,1.0,67.0,NaN,0.0,3.0,NaN,3.0,NaN,0.0,10.0,NaN,2.0,1.0,2.0,0.0,0.0,0.0,0.0,20150.0,2.0,2.0,0.0,4.0,NaN,6.0,600.0,49 Stat. 801,87.0,87.0,1,5,3
4,1946-005,1946-005-01,1946-005-01-01,1946-005-01-01-01,11/25/1946,1,329 U.S. 64,67 S. Ct. 154,91 L. Ed. 44,1946 U.S. LEXIS 2997,1946,1301,Vinson,50,"UNITED STATES v. HOWARD P. FOLEY CO., INC.",10/25/1946,NaN,27,NaN,176.0,NaN,1.0,NaN,NaN,0.0,3.0,NaN,3.0,NaN,0.0,2.0,NaN,2.0,1.0,3.0,0.0,1.0,0.0,0.0,80060.0,8.0,2.0,0.0,7.0,NaN,NaN,NaN,NaN,78.0,87.0,1,6,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9090,2020-070,2020-070-01,2020-070-01-01,2020-070-01-01-01,3/29/2021,2,NaN,141 S. Ct. 1145,209 L. Ed. 2d 265,2021 U.S. LEXIS 1736,2020,1709,Roberts,20-507,"MAYS, WARDEN v. HINES",NaN,NaN,7,50.0,215.0,NaN,1.0,NaN,NaN,0.0,117.0,NaN,26.0,NaN,1.0,12.0,3.0,2.0,1.0,3.0,0.0,1.0,0.0,0.0,10020.0,1.0,1.0,0.0,4.0,NaN,3.0,341.0,NaN,NaN,111.0,1,8,1
9091,2020-071,2020-071-01,2020-071-01-01,2020-071-01-01-01,4/26/2021,2,NaN,141 S. Ct. 1467,209 L. Ed. 2d 431,2021 U.S. LEXIS 2224,2020,1709,Roberts,20-940,ALASKA v. WRIGHT,NaN,NaN,28,2.0,126.0,NaN,1.0,NaN,NaN,0.0,44.0,NaN,29.0,NaN,0.0,12.0,4.0,2.0,1.0,5.0,0.0,1.0,0.0,0.0,10020.0,1.0,1.0,NaN,4.0,3.0,3.0,341.0,NaN,NaN,111.0,1,9,0
9092,2020-072,2020-072-01,2020-072-01-01,2020-072-01-01-01,7/2/2021,2,NaN,141 S. Ct. 2405,210 L. Ed. 2d 812,2021 U.S. LEXIS 3590,2020,1709,Roberts,20-1084,DUNN v. REEVES,NaN,NaN,7,1.0,215.0,NaN,1.0,NaN,NaN,0.0,43.0,NaN,31.0,NaN,0.0,12.0,6.0,2.0,1.0,4.0,0.0,1.0,0.0,0.0,10020.0,1.0,1.0,0.0,3.0,NaN,3.0,341.0,NaN,NaN,111.0,1,6,3
9093,2020-073,2020-073-01,2020-073-01-01,2020-073-01-01-01,6/28/2021,2,NaN,141 S. Ct. 2226,210 L. Ed. 2d 617,2021 U.S. LEXIS 3557,2020,1709,Roberts,20-1212,PAKDEL v. CITY AND COUNTY OF SAN FRANCISCO,NaN,NaN,195,NaN,3.0,6.0,1.0,NaN,NaN,0.0,50.0,NaN,29.0,NaN,1.0,12.0,2.0,2.0,3.0,5.0,0.0,

(9095, 53)

In [4]:
print(cases.columns.get_loc("decisionDirection"))
cases.shape

41


(9095, 53)

In [5]:
#print(cases['chief'].unique());
cases.shape

(9095, 53)

In [6]:
cases = cases[cases['decisionDirection'].notna()]

In [7]:
cases.shape

(9055, 53)

In [8]:
pct_null = cases.isnull().sum() / len(cases)
print(pct_null)
missing_features = pct_null[pct_null > 0.05].index
cases.drop(missing_features, axis=1, inplace=True)


caseId                      0.000000
docketId                    0.000000
caseIssuesId                0.000000
voteId                      0.000000
dateDecision                0.000000
decisionType                0.000000
usCite                      0.045941
sctCite                     0.000000
ledCite                     0.000110
lexisCite                   0.000000
term                        0.000000
naturalCourt                0.000000
chief                       0.000000
docket                      0.000883
caseName                    0.000000
dateArgument                0.116621
dateRearg                   0.980453
petitioner                  0.000000
petitionerState             0.795583
respondent                  0.000000
respondentState             0.719934
jurisdiction                0.000110
adminAction                 0.726449
adminActionState            0.926781
threeJudgeFdc               0.001767
caseOrigin                  0.035340
caseOriginState             0.725566
c

In [9]:
cases.shape

(9055, 39)

In [10]:
cases

,caseId,docketId,caseIssuesId,voteId,dateDecision,decisionType,usCite,sctCite,ledCite,lexisCite,term,naturalCourt,chief,docket,caseName,petitioner,respondent,jurisdiction,threeJudgeFdc,caseOrigin,caseSource,lcDisagreement,certReason,lcDispositionDirection,declarationUncon,caseDisposition,caseDispositionUnusual,partyWinning,precedentAlteration,voteUnclear,issue,issueArea,decisionDirection,decisionDirectionDissent,authorityDecision1,majOpinAssigner,splitVote,majVotes,minVotes
0,1946-001,1946-001-01,1946-001-01-01,1946-001-01-01-01,11/18/1946,1,329 U.S. 1,67 S. Ct. 6,91 L. Ed. 3,1946 U.S. LEXIS 1724,1946,1301,Vinson,24,HALLIBURTON OIL WELL CEMENTING CO. v. WALKER e...,198,172.0,6.0,0.0,51.0,29.0,0.0,11.0,1.0,1.0,3.0,0.0,1.0,1.0,0.0,80180.0,8.0,2.0,0.0,4.0,78.0,1,8,1
1,1946-002,1946-002-01,1946-002-01-01,1946-002-01-01-01,11/18/1946,1,329 U.S. 14,67 S. Ct. 13,91 L. Ed. 12,1946 U.S. LEXIS 1725,1946,1301,Vinson,12,CLEVELAND v. UNITED STATES,100,27.0,1.0,0.0,123.0,30.0,0.0,4.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0,10500.0,1.0,1.0,0.0,4.0,87.0,1,6,3
2,1946-003,1946-003-01,1946-003-01-01,1946-003-01-01-01,11/18/1946,1,329 U.S. 29,67 S. Ct. 1,91 L. Ed. 22,1946 U.S. LEXIS 3037,1946,1301,Vinson,21,CHAMPLIN REFINING CO. v. UNITED STATES ET AL.,209,27.0,2.0,1.0,107.0,107.0,0.0,1.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0,80250.0,8.0,2.0,0.0,1.0,78.0,1,5,4
3,1946-004,1946-004-01,1946-004-01-01,1946-004-01-01-01,11/25/1946,7,329 U.S. 40,67 S. Ct. 167,91 L. Ed. 29,1946 U.S. LEXIS 1696,1946,1301,Vinson,26,UNITED STATES v. ALCEA BAND OF TILLAMOOKS ET AL.,27,170.0,1.0,0.0,3.0,3.0,0.0,10.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0,20150.0,2.0,2.0,0.0,4.0,87.0,1,5,3
4,1946-005,1946-005-01,1946-005-01-01,1946-005-01-01-01,11/25/1946,1,329 U.S. 64,67 S. Ct. 154,91 L. Ed. 44,1946 U.S. LEXIS 2997,1946,1301,Vinson,50,"UNITED STATES v. HOWARD P. FOLEY CO., INC.",27,176.0,1.0,0.0,3.0,3.0,0.0,2.0,2.0,1.0,3.0,0.0,1.0,0.0,0.0,80060.0,8.0,2.0,0.0,7.0,87.0,1,6,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9090,2020-070,2020-070-01,2020-070-01-01,2020-070-01-01-01,3/29/2021,2,NaN,141 S. Ct. 1145,209 L. Ed. 2d 265,2021 U.S. LEXIS 1736,2020,1709,Roberts,20-507,"MAYS, WARDEN v. HINES",7,215.0,1.0,0.0,117.0,26.0,1.0,12.0,2.0,1.0,3.0,0.0,1.0,0.0,0.0,10020.0,1.0,1.0,0.0,4.0,111.0,1,8,1
9091,2020-071,2020-071-01,2020-071-01-01,2020-071-01-01-01,4/26/2021,2,NaN,141 S. Ct. 1467,209 L. Ed. 2d 431,2021 U.S. LEXIS 2224,2020,1709,Roberts,20-940,ALASKA v. WRIGHT,28,126.0,1.0,0.0,44.0,29.0,0.0,12.0,2.0,1.0,5.0,0.0,1.0,0.0,0.0,10020.0,1.0,1.0,NaN,4.0,111.0,1,9,0
9092,2020-072,2020-072-01,2020-072-01-01,2020-072-01-01-01,7/2/2021,2,NaN,141 S. Ct. 2405,210 L. Ed. 2d 812,2021 U.S. LEXIS 3590,2020,1709,Roberts,20-1084,DUNN v. REEVES,7,215.0,1.0,0.0,43.0,31.0,0.0,12.0,2.0,1.0,4.0,0.0,1.0,0.0,0.0,10020.0,1.0,1.0,0.0,3.0,111.0,1,6,3
9093,2020-073,2020-073-01,2020-073-01-01,2020-073-01-01-01,6/28/2021,2,NaN,141 S. Ct. 2226,210 L. Ed. 2d 617,2021 U.S. LEXIS 3557,2020,1709,Roberts,20-1212,PAKDEL v. CITY AND COUNTY OF SAN FRANCISCO,195,3.0,1.0,0.0,50.0,29.0,1.0,12.0,2.0,3.0,5.0,0.0,1.0,0.0,0.0,40070.0,4.0,1.0,NaN,2.0,111.0,1,9,0


### Data Cleaning and Custom Transformer

In [11]:
from sklearn.model_selection import train_test_split
x = cases.drop('decisionDirection',axis=1)
y = cases['decisionDirection']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [12]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((7244, 38), (7244,), (1811, 38), (1811,))

In [13]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7244 entries, 5384 to 2762
Data columns (total 38 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   caseId                    7244 non-null   object 
 1   docketId                  7244 non-null   object 
 2   caseIssuesId              7244 non-null   object 
 3   voteId                    7244 non-null   object 
 4   dateDecision              7244 non-null   object 
 5   decisionType              7244 non-null   int64  
 6   usCite                    6909 non-null   object 
 7   sctCite                   7244 non-null   object 
 8   ledCite                   7244 non-null   object 
 9   lexisCite                 7244 non-null   object 
 10  term                      7244 non-null   int64  
 11  naturalCourt              7244 non-null   int64  
 12  chief                     7244 non-null   object 
 13  docket                    7239 non-null   object 
 14  caseN

In [14]:
# drop columns with nulls of greater than 5% 
# check all cols are correct data type
# replace ordinal with one hot
"""
use ordinal data if there's some kind of order to the data you're using

ordinal:
A = 1
B = 2
C = 3

one hot
A = [1, 0, 0]
B = [0, 1, 0]
C = [0, 0, 1]
"""

"\nuse ordinal data if there's some kind of order to the data you're using\n\nordinal:\nA = 1\nB = 2\nC = 3\n\none hot\nA = [1, 0, 0]\nB = [0, 1, 0]\nC = [0, 0, 1]\n"

Vinson - [1, 0, 0, 0, 0]

Warren - [0, 1, 0, 0, 0]

In [15]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

numeric_transformer = Pipeline(steps=[
       ('imputer', SimpleImputer(strategy='mean'))
      ,('scaler', StandardScaler())
])
categorical_transformer = Pipeline(steps=[
       ('imputer', SimpleImputer(strategy='constant'))
      ,('encoder', OneHotEncoder())
])

In [16]:
numeric_features = ['decisionType', 'term', 'naturalCourt', 'petitioner', 'respondent', 'jurisdiction', 'threeJudgeFdc', 'caseOrigin', 'caseSource', 'lcDisagreement', 'certReason', 'lcDispositionDirection','declarationUncon','caseDisposition','caseDispositionUnusual','partyWinning','precedentAlteration','voteUnclear','issue','issueArea','authorityDecision1','majOpinAssigner','splitVote','majVotes','minVotes']
categorical_features = ['docket','usCite','sctCite','ledCite','lexisCite','chief','caseName', 'dateDecision','caseIssuesId','voteId']
preprocessor = ColumnTransformer(
   transformers=[
    ('numeric', numeric_transformer, numeric_features)
   ,('categorical', categorical_transformer, categorical_features)
]) 

## Models & Predictions

### Logistic Regression Model

In [17]:
from sklearn.linear_model import LogisticRegression
import numpy as np 

lm = Pipeline(steps = [
               ('preprocessor', preprocessor)
              ,('regressor', LogisticRegression())
           ])

In [19]:
lm.fit(x_train, y_train)

#prediction
y_pred1 = lm.predict(x_test)

#call fit transform to test the pipeline
#streamlit to make an app for the model
#logisitc, svc, random forest, decision tree, extra tree, lgbm
#optional: run a grid search to find the best hyperparameters
# test multiple models and grid search them

/shared-libs/python3.8/py/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


ValueError: Found unknown categories ['00-767', '12-414', '89-213', '09-940', '76-1206', '92-6281', '05-705', '85-766', '08-598', '73-781', '73-5284', '85-664', '84-5332', '74-5116', '78-759', '09-1272', '93-1286', '82-1554', '91-17', '88-599', '72-888', '74-1151', '81-1064', '84-237', '74-6738', '94-8769', '81-341', '89-5383', '254', '79-244', '92-1450', '76-5856', '79-1', '01-704', '86-393', '562', '03-9627', '13-298', '80-1002', '80-2195', '79-678', '13-975', '72-1057', '75-1906', '73-582', '07-1216', '84-1644', '962', '99-1331', '70-31', '85-621', '83-181', '15-8049', '91-594', '03-814', '72-1396', '83-1416', '82-862', '04-1244', '81-1664', '72-702', '86-1381', '935', '01-1229', '73-6848', '07-10689', '88-2035', '70-61', '16-6795', '16-327', '87-336', '93-284', '89-624', '90-96', '598', '75-543', '91-1502', '75-6933', '07-212', '90-5193', '86-1940', '92-351', '89-1838', '172', '85-1140', '96-1971', '78-80', '85-5487', '17-459', '08-214', '75-1126', '82-1271', '264', '82-945', '77-7

In [19]:
print("Accuracy: ",metrics.accuracy_score(y_test,y_pred1))

NameError: name 'y_pred' is not defined

### KNN

### Random Forest

In [25]:
from sklearn.ensemble import RandomForestClassifier
pipeline = Pipeline(steps = [
               ('preprocessor', preprocessor)
              ,('classifier', RandomForestClassifier(n_estimators=100))
           ])

rfm = pipeline.fit(x_train,y_train)
y_pred = pipeline.predict(x_test)
print("Accuracy: ", metrics.accuracy_score(y_test,y_pred))


ValueError: Found unknown categories ['89-1166', '835', '79-1601', '5272', '409', '79-1890', '05-128', '77-1387', '06-457', '15-1358', '73-6033', '74-895', '74-1435', '14-114', '00-1089', '94-1175', '84-1103', '99-1434', '75-5952', '13-719', '95-1228', '00-9280', '71-452', '75-5014', '206M', '76-1005', '83-2004', '12-872', '18-459', '74-201', '18-6662', '499', '97-826', '76-6513', '03-9627', '95-1263', '83-2146', '93-1783', '08-651', '87-336', '96-1375', '91-10', '81-271', '93-9220', '95-1225', '531', '80-148', '77-308', '85-1088', '01-595', '85-766', '89-386', '78-759', '06-5618', '17-1678', '78-680', '05-9264', '12-142', '08-1119', '97-634', '90-1074', '94-898', '08-1234', '07-562', '00-152', '57 ORIG', '14-614', '87-59', '90-5193', '84-589', '08-22', '98-231', '92-1639', '86-1431', '5247', '92-479', '81-17', '78-5944', '04-1327', '13-1041', '85-488', '88-2035', '86-2037', '77-753', '89-5383', '72-1040', '17-949', '83-96', '13-1032', '72-702', '81-2159', '84-648', '935', '09-940', '1

In [ ]:
# numerical cols
'caseId','docketId','caseIssuesId','voteId','dateDecision','decisionType','term','naturalCourt','docket','dateArgument','dateRearg','petitioner','petitionerState','respondent','respondentState','jurisdiction','adminAction','adminActionState','threeJudgeFdc','caseOrigin','caseOriginState','caseSource','caseSourceState','lcDisagreement','certReason','lcDisposition','lcDispositionDirection','declarationUncon','caseDisposition','caseDispositionUnusual','partyWinning','precedentAlteration','voteUnclear','issue','issueArea','decisionDirection','decisionDirectionDissent','authorityDecision1','authorityDecision2','lawType','lawSupp', 'majOpinWriter','majOpinAssigner','splitVote','majVotes','minVotes'

In [ ]:
# categorical cols
'usCite','sctCite','ledCite','lexisCite','chief','caseName','lawMinor'

In [ ]:
cases.shape

(9095, 53)

In [ ]:
"""
dateDecision: idx = 1
caseName: idx = 2
issueArea: idx = 3
authorityDecision: idx = 4
lawType: idx = 5
lcDisposition: idx = 6
lcDispositionDirection: idx = 7
chief: idx = 8
caseOrigin: idx = 9

Chief Justices:
- Vinson
- Warren
- Burger
- Rehnquist
- Roberts
"""

'\ndateDecision: idx = 1\ncaseName: idx = 2\nissueArea: idx = 3\nauthorityDecision: idx = 4\nlawType: idx = 5\nlcDisposition: idx = 6\nlcDispositionDirection: idx = 7\nchief: idx = 8\ncaseOrigin: idx = 9\n\nChief Justices:\n- Vinson\n- Warren\n- Burger\n- Rehnquist\n- Roberts\n'

## Feature Selection and Splitting Data

## Linear Regression

## Logistic Regression

## KNN

## Feature Selection

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

X = cases.iloc[:,0:52]  #independent columns
y = cases.iloc[:,40]    #target column i.e price range

#apply SelectKBest class to extract top 10 best features
bestfeatures = SelectKBest(score_func=chi2, k=10)
fit = bestfeatures.fit(X,y)

"""
dfscores = cases.DataFrame(fit.scores_)
dfcolumns = cases.DataFrame(X.columns)
#concat two dataframes for better visualization 
featureScores = cases.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']  #naming the dataframe columns
print(featureScores.nlargest(10,'Score'))  #print 10 best features
"""

ValueError: could not convert string to float: '1946-001'

In [ ]:
#print(cases.shape)
#print(cases.iloc[0])
independent_col = cases.iloc[:,0:52]
target_col = cases.iloc[40]

from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt
model = ExtraTreesClassifier()
model.fit(independent_col, target_col)
print(model.feature_importances_) #use inbuilt class feature_importances of tree based classifiers
#plot graph of feature importances for better visualization
feat_importances = pd.Series(model.feature_importances_, index=independent_col.columns)
feat_importances.nlargest(10).plot(kind='barh')
plt.show()

ValueError: could not convert string to float: '1946-001'

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=bce6a339-bf53-4d01-a32c-ce5d04f37b23' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>